In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import sys
sys.path.append('../')

In [6]:
from assesssample import AssessVarCat, AssessVarCat, AssessCombVar

In [78]:
import numpy as np
import pandas as pd
from collections import Counter

In [2]:
data = pd.read_csv('../data/iris.csv')

In [3]:
data.head()

,sepal_length,sepal_width,petal_length,petal_width,iris_class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


## stratified sample
* evaluate target class proportions
* get random n_sample*proportion_classes
* check combVar proba

In [8]:
class_prop = {x:len(data[data.iris_class == x])/len(data) for x in data.iris_class.unique()}
class_prop

{'Iris-setosa': 0.3333333333333333,
 'Iris-versicolor': 0.3333333333333333,
 'Iris-virginica': 0.3333333333333333}

In [9]:
n_samples = 20

In [10]:
traget_prop = {k: round(v * n_samples) for k,v in class_prop.items()}

In [11]:
traget_prop

{'Iris-setosa': 7, 'Iris-versicolor': 7, 'Iris-virginica': 7}

Sample according to the frequency distribution

In [84]:
c = data[data.iris_class == 'Iris-setosa']

In [13]:
len(c)

50

In [17]:
freq_bin, bin_edges = np.histogram(c.sepal_length, density=True)

In [31]:
counts_bin = np.round(freq_bin/np.sum(freq_bin) * traget_prop['Iris-setosa'])

In [38]:
counts_bin

array([1., 0., 1., 1., 2., 1., 1., 1., 0., 0.])

In [32]:
bin_edges

array([4.3 , 4.45, 4.6 , 4.75, 4.9 , 5.05, 5.2 , 5.35, 5.5 , 5.65, 5.8 ])

In [54]:
bin_edges_windows = np.array([bin_edges[0:len(bin_edges)-1], bin_edges[1:len(bin_edges)]])
bin_edges_windows

array([[4.3 , 4.45, 4.6 , 4.75, 4.9 , 5.05, 5.2 , 5.35, 5.5 , 5.65],
       [4.45, 4.6 , 4.75, 4.9 , 5.05, 5.2 , 5.35, 5.5 , 5.65, 5.8 ]])

In [61]:
np.arange(len(counts_bin))[counts_bin.astype(bool)]

array([0, 2, 3, 4, 5, 6, 7])

In [69]:
counts_bin[counts_bin.astype(bool)].astype(int)

array([1, 1, 1, 2, 1, 1, 1])

In [71]:
steps = np.repeat(np.arange(len(counts_bin))[counts_bin.astype(bool)], counts_bin[counts_bin.astype(bool)].astype(int))

In [77]:
indexes = np.hstack([np.where((c.sepal_length > bin_edges_windows[0,i]) & (c.sepal_length <= bin_edges_windows[1,i])) for i in steps])

In [302]:
def get_indices(column, n_samples):
    """get most probable indices samples according to the column values distribution"""
    freq_bin, bin_edges = np.histogram(column, density=True)
    # get the number of samples in each bins of the distribution
    counts_bin = np.round(freq_bin/np.sum(freq_bin) * n_samples)
    bin_edges_windows = np.array([bin_edges[0:len(bin_edges)-1], bin_edges[1:len(bin_edges)]])

    steps = np.repeat(np.arange(len(counts_bin))[counts_bin > 0.], counts_bin[counts_bin > 0.].astype(int))
    indexes = [column.index.where((column > bin_edges_windows[0,i]) & 
                                  (column <= bin_edges_windows[1,i])).tolist() for i in steps]

    return [x for sublist in indexes for x in sublist if not np.isnan(x)]

In [303]:
i_sl = get_indices(data[data['iris_class'] == 'Iris-versicolor'].sepal_length, n_samples)

In [304]:
i_sl

[60.0,
 93.0,
 98.0,
 60.0,
 93.0,
 98.0,
 53.0,
 80.0,
 81.0,
 84.0,
 89.0,
 90.0,
 53.0,
 80.0,
 81.0,
 84.0,
 89.0,
 90.0,
 55.0,
 64.0,
 66.0,
 69.0,
 79.0,
 88.0,
 94.0,
 95.0,
 96.0,
 99.0,
 55.0,
 64.0,
 66.0,
 69.0,
 79.0,
 88.0,
 94.0,
 95.0,
 96.0,
 99.0,
 55.0,
 64.0,
 66.0,
 69.0,
 79.0,
 88.0,
 94.0,
 95.0,
 96.0,
 99.0,
 55.0,
 64.0,
 66.0,
 69.0,
 79.0,
 88.0,
 94.0,
 95.0,
 96.0,
 99.0,
 61.0,
 67.0,
 70.0,
 82.0,
 92.0,
 61.0,
 67.0,
 70.0,
 82.0,
 92.0,
 62.0,
 63.0,
 71.0,
 73.0,
 78.0,
 83.0,
 85.0,
 91.0,
 62.0,
 63.0,
 71.0,
 73.0,
 78.0,
 83.0,
 85.0,
 91.0,
 62.0,
 63.0,
 71.0,
 73.0,
 78.0,
 83.0,
 85.0,
 91.0,
 56.0,
 68.0,
 72.0,
 87.0,
 97.0,
 56.0,
 68.0,
 72.0,
 87.0,
 97.0,
 51.0,
 54.0,
 74.0,
 58.0,
 65.0,
 75.0,
 77.0,
 86.0,
 58.0,
 65.0,
 75.0,
 77.0,
 86.0,
 50.0,
 52.0,
 76.0]

In [261]:
np.where((data[data['iris_class'] == 'Iris-versicolor'].sepal_length > 4))

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]),)

In [282]:
data[data['iris_class'] == 'Iris-versicolor'].sepal_length.index.where(data[data['iris_class'] == 'Iris-versicolor'].sepal_length >4).tolist()

[50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99]

In [184]:
i_sw = get_indices(c.sepal_width, n_samples)

In [185]:
i_sw

array([ 1,  3,  9, 12, 13, 25, 30, 34, 37, 38, 45,  1,  3,  9, 12, 13, 25,
       30, 34, 37, 38, 45,  1,  3,  9, 12, 13, 25, 30, 34, 37, 38, 45,  1,
        3,  9, 12, 13, 25, 30, 34, 37, 38, 45,  2, 23, 29, 35, 42, 47, 49,
        2, 23, 29, 35, 42, 47, 49,  2, 23, 29, 35, 42, 47, 49,  0,  6,  7,
       11, 17, 20, 24, 26, 27, 28, 31, 36, 39, 40, 43,  0,  6,  7, 11, 17,
       20, 24, 26, 27, 28, 31, 36, 39, 40, 43,  0,  6,  7, 11, 17, 20, 24,
       26, 27, 28, 31, 36, 39, 40, 43,  0,  6,  7, 11, 17, 20, 24, 26, 27,
       28, 31, 36, 39, 40, 43,  0,  6,  7, 11, 17, 20, 24, 26, 27, 28, 31,
       36, 39, 40, 43,  0,  6,  7, 11, 17, 20, 24, 26, 27, 28, 31, 36, 39,
       40, 43,  4, 10, 21, 22, 48,  4, 10, 21, 22, 48,  5, 16, 18, 19, 44,
       46,  5, 16, 18, 19, 44, 46, 14, 32, 15, 33])

In [151]:
indices = np.hstack([get_indices(c[x], n_samples) for x in ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])

In [152]:
sample_choices = []
while len(sample_choices) < traget_prop['Iris-setosa']:
    x = np.random.choice(indices, size=1)[0]
    if x not in sample_choices:
        sample_choices.append(x)

In [153]:
sample_choices

[27, 26, 34, 3, 32, 19, 0]

In [154]:
data.iloc[sample_choices,:]

,sepal_length,sepal_width,petal_length,petal_width,iris_class
27,5.2,3.5,1.5,0.2,Iris-setosa
26,5.0,3.4,1.6,0.4,Iris-setosa
34,4.9,3.1,1.5,0.1,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
32,5.2,4.1,1.5,0.1,Iris-setosa
19,5.1,3.8,1.5,0.3,Iris-setosa
0,5.1,3.5,1.4,0.2,Iris-setosa


In [129]:
cols = list(data.columns)

In [131]:
cols.remove('iris_class')

In [132]:
cols

['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

In [202]:
for k,v in traget_prop.items():
    print(data[data['iris_class'] == k].head())

   sepal_length  sepal_width  petal_length  petal_width   iris_class
0           5.1          3.5           1.4          0.2  Iris-setosa
1           4.9          3.0           1.4          0.2  Iris-setosa
2           4.7          3.2           1.3          0.2  Iris-setosa
3           4.6          3.1           1.5          0.2  Iris-setosa
4           5.0          3.6           1.4          0.2  Iris-setosa
    sepal_length  sepal_width  petal_length  petal_width       iris_class
50           7.0          3.2           4.7          1.4  Iris-versicolor
51           6.4          3.2           4.5          1.5  Iris-versicolor
52           6.9          3.1           4.9          1.5  Iris-versicolor
53           5.5          2.3           4.0          1.3  Iris-versicolor
54           6.5          2.8           4.6          1.5  Iris-versicolor
     sepal_length  sepal_width  petal_length  petal_width      iris_class
100           6.3          3.3           6.0          2.5  Iris-virg

In [305]:
# for each class
def stratified_sample(data, n_samples, target_col):
    sample_idx = []
    cols = list(data.columns)
    cols.remove(target_col)
    class_prop = {x:len(data[data[target_col] == x])/len(data) for x in data[target_col].unique()}
    target_prop = {k: round(v * n_samples) for k,v in class_prop.items()}
    
    for k,v in target_prop.items():
        
        data_stratum = data[data[target_col] == k]
        
        indices = np.hstack([get_indices(data_stratum[x], v) for x in cols])
        sample_choices = []
        while len(sample_choices) < v:
            x = np.random.choice(indices, size=1)[0]
            if x not in sample_choices:
                sample_choices.append(x)
        
        sample_idx.extend(sample_choices)
    return sample_idx

In [358]:
idx = stratified_sample(data, 5, 'iris_class')
idx

[17.0, 39.0, 53.0, 88.0, 139.0, 141.0]

In [397]:
AssessCombVar(data.iloc[idx,:], data, 'iris_class').evaluate()

0.999978575154218

In [388]:
AssessCombVar(data.sample(20), data, 'iris_class').evaluate()

0.9915566165778669

In [360]:
AssessCombVar(data.iloc[idx,:], data, 'iris_class').evaluate()

1.0

In [375]:
AssessCombVar(data[~data.index.isin(idx)], data, 'iris_class').evaluate()

1.0